## AWS y Intel Hackathon: Model Training

### Install Python SDKs

In [1]:
import sys

In [2]:
!{sys.executable} -m pip install sagemaker-experiments==0.1.24

### Install PyTroch

In [3]:
!{sys.executable} -m pip install torch==1.1.0
!{sys.executable} -m pip install torchvision==0.3.0
!{sys.executable} -m pip install pillow==6.2.2
!{sys.executable} -m pip install --upgrade sagemaker
!{sys.executable} -m pip install torchsummary

     |████████████████████████████████| 676.9 MB 1.9 kB/s             
     |████████████████████████████████| 2.6 MB 27.7 MB/s            
     |████████████████████████████████| 2.1 MB 26.0 MB/s            
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bokeh 2.3.3 requires pillow>=7.1.0, but you have pillow 6.2.2 which is incompatible.
     |████████████████████████████████| 522 kB 22.6 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.87.0-py2.py3-none-any.whl size=721735 sha256=e903f726a02f8a49ffa1b970ee817c539902b8f2093ea29ab2bc08ff5c21d794
  Stored in directory: /home/ec2-user/.cache/pip/wheels/0a/d2/c0/292d072dd8cd79b2cbbb1e40502629e45ce86b44e4

### Setup

In [4]:
import time

import boto3
import numpy as np
import pandas as pd
from IPython.display import set_matplotlib_formats, display
from matplotlib import pyplot as plt
from torchvision import datasets, transforms, models

import torch

import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session

from tqdm.notebook import tqdm

from torchsummary import summary
import glob
from PIL import Image

import random

set_matplotlib_formats("retina")

### Download the data

In [5]:
!mkdir data

### Original dataset
Run the two cells below to use the original dataset. It should provide good results in training and test although the training process will take several minutes.

In [ ]:
!wget https://www.dropbox.com/s/zhljom0hth586p9/dataset_original.zip

In [ ]:
!mv dataset_original.zip data/dataset.zip
!unzip -quo data/dataset.zip -d data/

dataset_path = "./data/dataset.zip"

### Reduced Dataset
Run the two cells below to use the reduced dataset. It should provide worse results than the original but it will reduce the training process time.

In [ ]:
!wget https://www.dropbox.com/s/evm0ts2obk7n3cb/dataset_reduced.zip

In [ ]:
!mv dataset_reduced.zip data/dataset.zip
!unzip -quo data/dataset.zip -d data/

dataset_path = "./data/dataset.zip"


### Dataset_for_tests
Run the two cells below to use a very reduced dataset. It can be used for very fast tests although it will yield to very poor results in the predictions and training accuracy. 

In [6]:
!wget https://www.dropbox.com/s/zivlm0skt19k3wh/dataset_for_tests.zip

--2022-04-21 09:14:21--  https://www.dropbox.com/s/zivlm0skt19k3wh/dataset_for_tests.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6028:18::a27d:4712
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zivlm0skt19k3wh/dataset_for_tests.zip [following]
--2022-04-21 09:14:22--  https://www.dropbox.com/s/raw/zivlm0skt19k3wh/dataset_for_tests.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf77a09d255ca6f19baace4fd86.dl.dropboxusercontent.com/cd/0/inline/BjzHv6defNXDCrBCkUC-YzD-DXK5hXZSrJWLDyZe8ASyqg4e_qaEqq2TdRev2BEDRRw0c_BZkB7Qx4UgWNjeuyYi02F2SE8H9xBnHkBvLeUnTX6gntR5rJ7lTT9LwuYDKFwODK-4RoapLRH8NqjU87MbwM8APAu_akLakNYC76Trqw/file# [following]
--2022-04-21 09:14:22--  https://ucf77a09d255ca6f19baace4fd86.dl.dropboxusercontent.com/cd/0/inline/BjzHv6defNXDCrBCkUC-YzD-DXK5hXZSrJWLDy

In [7]:
!mv dataset_for_tests.zip data/dataset.zip
!unzip -quo data/dataset.zip -d data/

dataset_path = "./data/dataset.zip"

### Upload dataset to S3 as zip file

In [8]:
sm_sess = sagemaker.Session()
sess = sm_sess.boto_session
sm = sm_sess.sagemaker_client
role = get_execution_role()

In [9]:
account_id = sess.client("sts").get_caller_identity()["Account"]
bucket = "sagemaker-hackathon-demo-{}-{}".format(sess.region_name, account_id)
prefix = "hackathon"

try:
    if sess.region_name == "us-east-1":
        sess.client("s3").create_bucket(Bucket=bucket)
    else:
        sess.client("s3").create_bucket(
            Bucket=bucket, CreateBucketConfiguration={"LocationConstraint": sess.region_name}
        )
except Exception as e:
    print(e)

In [10]:
bucket

'sagemaker-hackathon-demo-eu-west-1-017233837209'

In [11]:
s3_resource = boto3.resource("s3", region_name = sess.region_name)

inputs = None

try:

    
    inputs = sagemaker.Session().upload_data(path=dataset_path, bucket=bucket, key_prefix=prefix)
    print("input spec: {}".format(inputs))
except Exception as exp:
    print("exp: ", exp)


input spec: s3://sagemaker-hackathon-demo-eu-west-1-017233837209/hackathon/dataset.zip


### Training

In [12]:
!pygmentize model.py

import argparse
import json
import logging
import os
import sys

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

import boto3
import zipfile
from torch.utils.data import DataLoader

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

img_transform= {
    'dataset':
    transforms.Compose([
        transforms.Resize((28, 28)), 
        transforms.ToTensor(),
    ]),
    'training':
    transforms.Compose([
         transforms.RandomResizedCrop(size=224),
         transforms.RandomHorizontalFlip(),
    ]),
    'validation': 
    transforms.Compose([
        transforms.Resize(size=224),
    ]),
    'testing': 
    transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
    ]),

In [13]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="model.py",
    role=role,
    framework_version="1.4.0",
    py_version="py3",
    instance_count=2,
    instance_type="ml.m4.xlarge",
    hyperparameters={"epochs": 6, "backend": "gloo"},
)

In [14]:
estimator.fit({"training": inputs})

2022-04-21 09:14:36 Starting - Starting the training job...
2022-04-21 09:15:02 Starting - Preparing the instances for trainingProfilerReport-1650532475: InProgress
.........
2022-04-21 09:16:23 Downloading - Downloading input data......
2022-04-21 09:17:19 Training - Downloading the training image...
2022-04-21 09:18:04 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-04-21 09:18:07,655 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-04-21 09:18:07,658 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-04-21 09:18:07,674 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-04-21 09:18:07,678 sagemaker_pytorch_container.trai

In [16]:
estimator.model_data

's3://sagemaker-eu-west-1-017233837209/pytorch-training-2022-04-21-09-14-35-467/output/model.tar.gz'

### Deployment

In [17]:
estimator.model_data

's3://sagemaker-eu-west-1-017233837209/pytorch-training-2022-04-21-09-14-35-467/output/model.tar.gz'

First we create a folder to save model trained model, and download the model.tar.gz file to local directory.

In [18]:
%%sh -s $estimator.model_data
mkdir model
aws s3 cp $1 model/ 
tar xvzf model/model.tar.gz --directory ./model

download: s3://sagemaker-eu-west-1-017233837209/pytorch-training-2022-04-21-09-14-35-467/output/model.tar.gz to model/model.tar.gz
model.pth
model.pth


Convert your model into the TorchScript format using torch.jit.trace or torch.jit.script.

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 5)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model_loaded = torch.load("model/model.pth")
model = Net().to("cpu")
model = torch.nn.DataParallel(model)
model.load_state_dict(model_loaded)
print("completed")

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [20]:
import subprocess

trace_input = torch.rand(1, 3, 28, 28)
traced_model = torch.jit.trace(model.eval(), trace_input)

torch.jit.save(traced_model, "model.pth")
subprocess.call(["tar", "-czvf", "traced_hackathon_model.tar.gz", "model.pth"])

0

In [87]:
!pygmentize deploy_ei.py

# Copyright 2019-2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.
from __future__ import absolute_import

import logging
import os
import sys

import torch

import cv2
import numpy as np

import torch
import torchvision

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

# To use new EIA inference API, customer should use attach_eia(model, eia_ordinal_number)
VERSIONS_USE_NEW_API = ["1.5.1"]

def input_fn(input_data, cont

In [88]:
from sagemaker.pytorch import PyTorchModel
from datetime import datetime

instance_type = "ml.m4.xlarge"

# TorchScript model
tar_filename = "traced_hackathon_model.tar.gz"

# You can also upload model artifacts to S3
# print('Upload tarball to S3')
# model_data = sagemaker_session.upload_data(path=tar_filename, bucket=bucket, key_prefix=prefix)
model_data = tar_filename

endpoint_name = ("hackathon-ei-endpoint").replace(".", "").replace("_", "")

In [89]:
pytorch = PyTorchModel(
    model_data=model_data,
    role=role,
    entry_point="deploy_ei.py",
    framework_version="1.3.1",
    py_version="py3",
    sagemaker_session=sm_sess,
)

In [90]:
# Attach EI remotely

# Function will exit before endpoint is finished creating
predictor = pytorch.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    wait=True,
)


-------!

In [56]:
predictor

### Access the endpoint from outside for using real-time inference

In [91]:
import boto3

import json

import numpy as np

endpoint = 'hackathon-ei-endpoint'
 
runtime = boto3.Session().client('sagemaker-runtime')
 
# Read image into memory
with open("plastic171.jpg", 'rb') as f:
    payload = f.read()
    
print("type payload", type(payload))

type payload <class 'bytes'>


In [92]:
# Send image via InvokeEndpoint API
response = runtime.invoke_endpoint(EndpointName=endpoint, ContentType='application/x-image', Body=payload)
result = json.loads(response['Body'].read().decode())
result = np.array(result)
prediction = result.argmax(axis=1)[0]
print("class predicted", prediction + 1)


class predicted 5


### Delete endpoint

In [86]:
predictor.delete_endpoint()